In [36]:
links = [
'https://chinapost.nownews.com/20191030-824083',
'https://chinapost.nownews.com/20191030-824169',
'https://chinapost.nownews.com/20191108-841375',
'https://chinapost.nownews.com/20191122-867435',
'https://chinapost.nownews.com/20191130-878563',
'https://chinapost.nownews.com/20200108-924363',
'https://chinapost.nownews.com/20200109-928513',
'https://chinapost.nownews.com/20200201-945311',
'https://chinapost.nownews.com/20200407-1127319',
'https://chinapost.nownews.com/20200414-1159330',
'https://chinapost.nownews.com/20200515-1242218',
'https://chinapost.nownews.com/20200518-1258699',
'https://chinapost.nownews.com/20200520-1267474',
'https://chinapost.nownews.com/20200525-1289945',
'https://chinapost.nownews.com/20200526-1292278',
'https://chinapost.nownews.com/20200526-1293959',
'https://chinapost.nownews.com/20200528-1302880',
'https://chinapost.nownews.com/20200602-1337534',
'https://chinapost.nownews.com/20200602-1337612',
'https://chinapost.nownews.com/20200603-1337757',
'https://chinapost.nownews.com/20200604-1337302',
'https://chinapost.nownews.com/20200604-1337346',
'https://chinapost.nownews.com/20200604-1332800',
'https://chinapost.nownews.com/20200608-1349124',
'https://chinapost.nownews.com/20200611-1364863',
'https://chinapost.nownews.com/20200616-1390596',
'https://chinapost.nownews.com/20200703-1476935',
'https://chinapost.nownews.com/20200715-1530169',
'https://chinapost.nownews.com/20200717-1538568',
'https://chinapost.nownews.com/20200724-1579055',
'https://chinapost.nownews.com/20200730-1610732',
'https://chinapost.nownews.com/20200731-1616712',
'https://chinapost.nownews.com/20200812-1680491',
'https://chinapost.nownews.com/20200824-1697045',
'https://chinapost.nownews.com/20200826-1697347',
'https://chinapost.nownews.com/20200828-1697669',
'https://chinapost.nownews.com/20200907-1738298',
'https://chinapost.nownews.com/20201027-1836328',
'https://chinapost.nownews.com/20201113-1838623',
'https://chinapost.nownews.com/20201203-1887760',
'https://chinapost.nownews.com/20201230-1971363',
'https://chinapost.nownews.com/20210119-2024065',
'https://chinapost.nownews.com/20210205-2089294',
'https://chinapost.nownews.com/20210304-2173241',
'https://chinapost.nownews.com/20210312-2195744',
'https://chinapost.nownews.com/20210317-2208505',
'https://chinapost.nownews.com/20210325-2284854',
'https://chinapost.nownews.com/20210422-2388807',
'https://chinapost.nownews.com/20210423-2381801',
'https://chinapost.nownews.com/20210513-2469288',
'https://chinapost.nownews.com/20210517-2483518',
'https://chinapost.nownews.com/20210520-2495191',
'https://chinapost.nownews.com/20210521-2499732',
'https://chinapost.nownews.com/20210526-2517850',
'https://chinapost.nownews.com/20210604-2550636',
'https://chinapost.nownews.com/20210610-2572301',
'https://chinapost.nownews.com/20210610-2567684'
]

In [37]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import fasttext

In [38]:
path_to_pretrained_model = './tmp/lid.176.bin'
fmodel = fasttext.load_model(path_to_pretrained_model)

In [39]:
def scrap_and_clean_text(link):
    source = urlopen(link).read()
    soup = BeautifulSoup(source,'lxml')
    text = ''
    for paragraph in soup.find_all('p'):
        text += paragraph.text
        
    text = re.sub(r'【.*?】+', '', text)
    text = re.sub(r"^\s+", "", text)
    text = re.sub(r"\xa0", "", text)
    text = re.sub(r"\u200b", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"Let’s look through the lens of a foreign friend to see Taiwan in a new light", "", text)
    text = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", "", text)

    
    rx = r'(?:(?<=[.!?。」！？：…])|(?<=[.!?]["”」！？：…]))'
#     rx = r'(?:(?<=[.,!?。」！？：…])|(?<=[.!?]["”」！？：…]))'
#     rx = r'(?:(?<=[.,!?。」！？：，…])|(?<=[.!?]["”」！？：，…]))'
    list_text = []

    for result in re.split(rx, text):
        list_text.append(result)
    
    for _ in range(20):
        for i in range(len(list_text)-1):
          lang1 = fmodel.predict(list_text[i])[0][0]
          lang2 = fmodel.predict(list_text[i+1])[0][0]
    
          if lang1 == '__label__ja':
            lang1 = '__label__zh'
    
          if lang1 == lang2:
            list_text[i] += list_text[i+1]
            list_text[i+1] = ''
        list_text = [x for x in list_text if not x == '']
    
    for _ in range(5):
        for i in range(1,len(list_text)):
            if fmodel.predict(list_text[i-1])[0][0] != '__label__zh':
              if len(list_text[i]) <=2:
                list_text[i-1] += list_text[i]
                list_text[i] = ''
        list_text = [x for x in list_text if not x == ''] 
    
    for _ in range(5):
        for i in range(len(list_text)):
            if len(list_text[i]) <=2:
                list_text[i-1] += list_text[i]
                list_text[i] = ''
        list_text = [x for x in list_text if not x == ''] 
        
    for _ in range(3):
        for i in range(len(list_text)-1):
          lang1 = fmodel.predict(list_text[i])[0][0]
          lang2 = fmodel.predict(list_text[i+1])[0][0]
    
          if lang1 == '__label__ja':
            lang1 = '__label__zh'
    
          if lang1 == lang2:
            list_text[i] += list_text[i+1]
            list_text[i+1] = ''
        list_text = [x for x in list_text if not x == '']
                
    for _ in range(10):
        for i in range(1,len(list_text)):
            if fmodel.predict(list_text[i])[0][0] != '__label__zh':
              if list_text[i][0] == ' ':
                list_text[i-1] += list_text[i]
                list_text[i] = ''
        list_text = [x for x in list_text if not x == ''] 
        
    for _ in range(3):
        for i in range(len(list_text)-1):
          lang1 = fmodel.predict(list_text[i])[0][0]
          lang2 = fmodel.predict(list_text[i+1])[0][0]
    
          if lang1 == '__label__ja':
            lang1 = '__label__zh'
            
          if lang2 == '__label__ja':
            lang2 = '__label__zh'
            
          if lang2 == '__label__nl':
            lang2 = '__label__en'
            
          if lang2 == '__label__wuu':
            lang2 = '__label__zh'
    
          if lang1 == lang2:
            list_text[i] += list_text[i+1]
            list_text[i+1] = ''
        list_text = [x for x in list_text if not x == '']
        
#     for _ in range(5):
#         for i in range(1,len(list_text)):
#             m = re.search(r"^\d+", list_text[i])
#             if m:
#                list_text[i-1] += list_text[i]
#                list_text[i] = ''
#         list_text = [x for x in list_text if not x == ''] 
        
    if fmodel.predict(list_text[len(list_text)-1])[0][0] == '__label__zh':
        list_text.pop(-1)
    
    en = []
    zh = []
    i=0

    while i <= len(list_text)-1:
        if i % 2 == 0:
            zh.append(list_text[i])
        else:
            en.append(list_text[i])
        i += 1
    
    return en, zh

In [40]:
en_all = []
zh_all = []

for link in links:
    en = []
    zh = []
    en, zh = scrap_and_clean_text(link)
    
    if len(en) == len(zh):
        en_all.extend(en)
        zh_all.extend(zh)

In [41]:
len(en_all)

405

In [42]:
len(zh_all)

405

In [43]:
en_all

['Do you want to travel the world and be a flight attendant? If that is the case, you should know that China Airlines (中華航空), the national carrier of Taiwan, said on Tuesday that it plans to hire 70 flight attendants next month.',
 'More information about recruitment will be announced between Nov. 1 and Nov. 14 on China Airlines’ official website, but you should already know that your first interview could be scheduled in late November.',
 'According to the company, cabin attendants are required to hold a bachelor’s degree or higher, a TOEIC test score of at least 600 or any equivalent English certification. If you are selected, you will join a 9-week long training program in Taiwan.',
 'If you haven’t made your mind about applying for the job, you should also know that China Airlines’ staff receives up to 123 days of annual leave, US$5 per hour for layover allowance, and other travel perks.',
 'The entry salary also ranges from NT$70,000 to NT$80,000.',
 'JR Hotel Group, one of the la

In [44]:
zh_all

['你想要成為空服員環遊世界嗎？如果這正是你期盼已久的夢想，那麼看這邊就對了，中華航空宣布下個月預計要招募70名空服員。',
 '更多相關資訊將於11 月 1 日至 14 日官方網站公開，初試時間預定在 11 月下旬。',
 '華航表示，本國籍客艙組員報考標準為具有教育部認可學士學位或以上者、多益 600 分或同等英語能力者；應試者通過甄試後將分梯受訓，接受為期 9 周的職前訓練。',
 '如果你還在考慮到底要不要應徵空服員，先來看看華航開出什麼福利吧！華航新進人員享保障年休 123 天、外站津貼每小時 5 美元、國定假日出勤給付加倍工資及員工優待機票等業界最優福利。',
 '上線後，每月平均收入約新台幣 7 至 8 萬元 (依實際派遣任務而定)。',
 '媒體報導， 日本規模最大的集團之一 JR 東日本集團傳正式接手台北威斯汀六福皇宮原建築，未來將以「 JR 東日本大飯店台北」營運。',
 '位在台北市南京東路三段的六福皇宮原址房東是國泰人壽。',
 '近期傳該建築將由 JR 東日集團接手承租，已與房東國泰人壽簽約，預計 11 月 6 日將會正式對外公告。',
 'JR 東日本集團在日本擁有多間飯店，包括東京、京都、大阪等都有據點。該集團飯店以常常提供持有日本鐵路通票（JR Pass）旅客折扣聞名。',
 '然而，JR 東日本集團不是唯一的日系集團想要進攻台北的飯店市場。',
 '台北多元文化及活潑的城市氛圍一直以來都吸引著很多日本觀光客造訪，所以日本集團想要爭取在台北開飯店也不足為奇。',
 '比如說日本西鐵集團將在台北鬧區西門町開設「Solaria西鐵飯店 – 台北西門」，預計於2023年開始營運。',
 '而三井花園飯店位於捷運忠孝新生站附近，將於明年開業；位在台北東區忠孝東路上的格拉斯麗飯店也預計在明年完工。',
 '至於其他已經在台北營運的日系飯店包括台北大倉久和大飯店、日勝生加賀屋國際溫泉飯店及台北燦路都飯店。',
 '來自北歐的家具品牌IKEA（宜家家居）臺灣發言人週四證實，IKEA要在台北內湖區開設臺灣的第七家分店。',
 '發言人吳語涵表示，IKEA內湖店不會是獨立店面，而是會進駐舊宗路上的內湖大潤發的複合式商場。',
 '舊宗路上以聚集大型商場如好市多、特力屋、家樂福等聞名，也有許多餐廳，因此該區一向是台北市民購物的首選區域。',
 '內湖店將

In [47]:
with open('english_ds_business2.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in en_all:
        filehandle.write('%s\n' % listitem)

In [48]:
with open('chinese_ds_business3.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in zh_all:
        filehandle.write('%s\n' % listitem)